Schritt 1: Alle nötigen Imports importieren

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

Schritt 2: Cuda und GPU als device festlegen

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Schritt 3: Implementierung des SNN und des LIF

In [6]:
# Beispiel LIF-Parameter und Zelle
class LIFParameters:
    def __init__(self):
        self.v_rest = -65  # Ruhepotenzial
        self.v_thresh = -50  # Schwellenwert
        self.v_reset = -65  # Reset-Potenzial
        self.tau_mem = 10  # Membranzeitkonstante

class LIFCell(nn.Module):
    def __init__(self, p):
        super(LIFCell, self).__init__()
        self.p = p
        self.v = None  # Membranpotential

    def forward(self, x):
        # Einfacher LIF-Zellenmechanismus, nur als Platzhalter
        spike_train = torch.randn_like(x) > 0  # Zufällige Spikes zur Demonstration
        return spike_train, None

class STDP_SNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01, tau_plus=20.0, tau_minus=20.0):
        super(STDP_SNN, self).__init__()
        # LIF-Parameter und Zellen
        lif_params = LIFParameters()
        self.lif1 = LIFCell(p=lif_params)
        self.synapse_weights = nn.Parameter(torch.randn(input_size, hidden_size) * 0.01)
        self.fc2 = nn.Linear(hidden_size, output_size)

        # Lernparameter
        self.learning_rate = learning_rate
        self.tau_plus = tau_plus
        self.tau_minus = tau_minus

    # STDP-Lernmethode
    def apply_stdp(self, prespike_time, postsynaptic_time):
        prespike_time = prespike_time[:, None]
        postsynaptic_time = postsynaptic_time[None, :]
        time_difference = postsynaptic_time - prespike_time
        weight_increase = torch.exp(-torch.abs(time_difference) / self.tau_plus)
        weight_decrease = torch.exp(-torch.abs(time_difference) / self.tau_minus)
        weight_delta = self.learning_rate * (weight_increase - weight_decrease)
        weight_delta[time_difference < 0] *= -1
        new_weights = torch.clamp(self.synapse_weights + weight_delta, min=0.0, max=1.0)
        self.synapse_weights = nn.Parameter(new_weights)

    # Normalisierung der Eingabedaten
    def normalize_input(self, data):
        std = data.std()
        return (data - data.mean()) / std if std > 0 else data - data.mean()

    # Vorwärtsdurchlauf
    def forward(self, x):
        x = self.normalize_input(x)
        spikes, _ = self.lif1(x)  # Extrahiere nur den Spike-Wert
        spikes_flat = spikes.view(spikes.size(0), -1)  # Flatten
        spikes_flat = spikes_flat[:, :self.fc2.in_features]  # Um auf die Anzahl der Features in fc2 zu passen
        
        # Umwandlung der Spikes in Float
        spikes_flat = spikes_flat.float()  # Umwandlung in float32 für den Linear-Layer
        
        output = self.fc2(spikes_flat)
        return output



Test (noch keine echten Daten)

In [7]:
# Transform für den MNIST-Datensatz
transform = transforms.Compose([
    transforms.ToTensor(),         # Konvertiere in Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalisierung
])

# Lade den MNIST-Datensatz
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def image_to_spikes(image, time_steps=100):
    batch_size = image.size(0)  # Correctly use the batch size from the input image
    spikes = torch.zeros(batch_size, time_steps, image.size(1))  # Shape: [Batch_Size, Time_Steps, Features]

    for t in range(time_steps):
        spike_train = (image > (t / time_steps)).float()  # Generate spike train based on pixel intensity
        spikes[:, t, :] = spike_train  # Set the spike train at the current time step
    
    return spikes


In [8]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 13 * 13, 10)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 32 * 13 * 13)
        x = self.fc1(x)
        return x

In [10]:
# Hyperparameter
input_size = 28 * 28
hidden_size = 100
output_size = 10
learning_rate = 0.001
epochs = 5
time_steps = 100

snn_model = STDP_SNN(input_size, hidden_size, output_size, learning_rate)
cnn_model = SimpleCNN().to(device)
snn_optimizer = torch.optim.Adam(snn_model.parameters(), lr=learning_rate)
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
# Function to average spikes over time
def average_spikes(spikes, time_steps):
    # Average spikes over the time dimension (e.g., by taking mean across time)
    return spikes.mean(dim=1)  # Shape: [batch_size, 784] after averaging over time

# Main training loop
for epoch in range(epochs):
    snn_model.train()
    cnn_model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.view(images.size(0), -1)  # Flatten the images
        spikes = image_to_spikes(images, time_steps)
        
        # Average over the time dimension before passing to CNN
        spikes_avg = average_spikes(spikes, time_steps).view(-1, 1, 28, 28)  # [batch_size, 1, 28, 28]

        # Now, you can pass the averaged spikes to both the SNN and CNN models
        outputs = snn_model(spikes)
        outputs2 = cnn_model(spikes_avg)

        loss = loss_fn(outputs, labels)

        # Backpropagation
        snn_optimizer.zero_grad()
        loss.backward()
        snn_optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    print(f'Epoch [{epoch+1}/{epochs}], SNN Loss: {running_loss/len(train_loader):.4f}')

# Testing (same modification)
snn_model.eval()
cnn_model.eval()
correct = 0
total = 0
correct2 = 0  # Initialisiere die Variable für die CNN-Ergebnisse

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        spikes = image_to_spikes(images, time_steps)
        spikes_avg = average_spikes(spikes, time_steps).view(-1, 1, 28, 28)  # [batch_size, 1, 28, 28]

        outputs = snn_model(spikes)
        outputs2 = cnn_model(spikes_avg)

        _, predicted = torch.max(outputs, 1)
        _, predicted2 = torch.max(outputs2, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        correct2 += (predicted2 == labels).sum().item()  # Diese Zeile wird jetzt funktionieren

print(f'Accuracy (SNN): {100 * correct / total:.2f}%')
print(f'Accuracy (CNN): {100 * correct2 / total:.2f}%')


Epoch [1/5], SNN Loss: 2.3242
Epoch [2/5], SNN Loss: 2.3103
Epoch [3/5], SNN Loss: 2.3082
Epoch [4/5], SNN Loss: 2.3081
Epoch [5/5], SNN Loss: 2.3084
Accuracy (SNN): 10.44%
Accuracy (CNN): 9.59%
